# Graph Neural Networks for Decentralized Multi-Robot Path Planning In Warehouse

### Importing the libraries

In [1]:
from Dataset_Generator import DatasetGenerator 
import numpy as np
import matplotlib.pyplot as plt

### Part 1 - Dataset Generation

In [ ]:
grid = [[0 for _ in range(20)] for _ in range(20)]  # Generate grid map
dataset_generator = DatasetGenerator(num_cases=5000, num_agents=10, grid=grid)
cases = dataset_generator.generate_cases()
# dataset_generator.save_cases_to_file(cases, "dataset.json")
# print(f"Generated and saved {len(cases)} cases.")

### Part 2 - Pre-Processing

In [ ]:
from pre_processing import Preprocessing
p = Preprocessing(grid,cases,3)
data_tensors  = p.begin()
with open("data_tensors.txt", 'w') as file:
    file.write(str(data_tensors))

In [ ]:
from Mo_Star import MoStar
from Dataset_Generator import DatasetGenerator
import numpy as np
grid = [[0 for _ in range(20)] for _ in range(20)] #generate grid map

dataset_generator = DatasetGenerator(5000,10,grid)
loaded_cases = dataset_generator.load_cases_from_file("dataset.json")
import networkx as nx

rows, cols = 20, 20
G = nx.grid_2d_graph(rows, cols)

percentages = []

for i in range(len(loaded_cases)):
    for j in range(len(loaded_cases[0]["start_positions"])):
        start = loaded_cases[i]["start_positions"][j]
        goal = loaded_cases[i]["goal_positions"][j]
        path = loaded_cases[i]["paths"][j]
        shortest_path = nx.shortest_path(G, source=(start[0],start[1]), target=(goal[0],goal[1]))
        percentages.append([len(shortest_path)/len(path)])
num_of_not_optimal_paths = 0

for i in range(len(percentages)):
    if percentages[i][0] != 1.0:
        num_of_not_optimal_paths+=1
avg_percentage = np.sum(percentages)/len(percentages)
print("Average percentages: ",avg_percentage)
print("Number of non optiaml paths: ", num_of_not_optimal_paths)
print("Percentage of non optimal paths to the all paths: ", num_of_not_optimal_paths/len(percentages))

Average percentages:  0.9943489490147263
Number of non optiaml paths:  2225
Percentage of non optimal paths to the all paths:  0.037083333333333336


In [ ]:
num_of_confilicts = 0
list_of_confilicts = []
i = 0
for case in loaded_cases:
    max_len = max(case["paths"])
    for step in range(len(max_len)):
        step_nodes = {}
        for path in case["paths"]:
            if step < len(path):
                if (path[step][0],path[step][1],step) in step_nodes:
                    num_of_confilicts+=1
                    list_of_confilicts.append((path[step][0],path[step][1],step,i))
                else:
                    step_nodes[(path[step][0],path[step][1],step)] = 1 
            else:
                if (path[-1][0],path[-1][1],step) in step_nodes:
                    num_of_confilicts+=1
                    list_of_confilicts.append((path[-1][0],path[-1][1],step,i))
                else:
                    step_nodes[(path[-1][0],path[-1][1],step)] = 1
        step_nodes.clear()
    i+=1
print(num_of_confilicts)
print(list_of_confilicts)

0
[]
